In [1]:
! nvidia-smi

Fri Aug 30 07:45:38 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        Off  | 00000000:00:05.0 Off |                  Off |
| 26%   33C    P8     9W / 180W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install -q llama-cpp-python==0.2.79

In [3]:
! pip install -q datasets==2.21.0 \
                 accelerate==0.33.0 \
                 evaluate==0.4.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.1.0 which is incompatible.


In [4]:
import numpy as np

from datasets import load_dataset

from llama_cpp import Llama

2024-08-30 08:01:19.754847: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-30 08:01:19.754925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-30 08:01:19.757364: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-30 08:01:19.786888: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-30 08:01:22.486339: W tensorflow/compiler/tf2

# Data

In [5]:
ds = load_dataset("ccdv/patent-classification", "abstract")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [6]:
train_data = ds['train']

In [7]:
test_data = ds['test']

# Prompt Design

In [8]:
llm = Llama.from_pretrained(
    repo_id="second-state/Meta-Llama-3.1-8B-Instruct-GGUF",
    filename="Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf",
    n_ctx=4096,
    n_cpu_layers=8,
    n_gpu_layers=48,
    verbose=True
)

Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 29 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--second-state--Meta-Llama-3.1-8B-Instruct-GGUF/snapshots/f24f6a255a71bd6211dccbc9bb67f83c51b7edab/./Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str              = 8B


In [9]:
llm.model_params.n_gpu_layers

48

In [10]:
llm.verbose = False

In [30]:
system_message = """
You are an expert assistant to a patent attorney tasked to classify patent abstracts into a fixed set of categories.
Each abstract can belong to only ONE of the following categories:
0: "Human Necessities", 
1: "Performing Operations; Transporting",
2: "Chemistry; Metallurgy",
3: "Textiles; Paper",
4: "Fixed Constructions",
5: "Mechanical Engineering; Lightning; Heating; Weapons; Blasting",
6: "Physics",
7: "Electricity",
8: "General tagging of new or cross-sectional technology"

A detailed description of each category is presented below.

Human Necessities: This category pertains to inventions that are related to fulfilling basic human needs. It includes patents for medical devices, pharmaceutical compositions, personal care products, food and beverages, clothing, and other inventions that directly impact human health, well-being, and daily living.
Physics: This category encompasses inventions that are related to the study of matter and energy. It includes patents for devices or processes that involve principles of mechanics, optics, acoustics, thermodynamics, quantum mechanics, and other areas of physics. Examples of patents in this category may include inventions related to lasers, semiconductors, optics, nuclear technology, and quantum computing.
Electricity: This category includes inventions that are related to the generation, transmission, distribution, and utilization of electrical energy. It covers patents for electrical circuits, power systems, electrical machinery and apparatus, electric vehicles, renewable energy technologies, and other electrical inventions.
General tagging of new or cross-sectional technology: This category is a catch-all for inventions that do not fit into any specific category but are related to new or emerging technologies. It includes patents for innovative technologies that may not have a specific field or application yet, but have the potential to disrupt various industries. Examples may include patents related to artificial intelligence, blockchain, virtual reality, augmented reality, and other cutting-edge technologies.
Performing Operations; Transporting: This category covers inventions related to processes, methods, and devices used in performing various operations or transporting goods, people, or information. It includes patents for manufacturing processes, industrial machinery, transportation vehicles, logistics systems, communication systems, and other inventions related to performing operations and transporting goods or information.
Chemistry; Metallurgy: This category includes inventions related to chemical processes, compositions, and materials. It covers patents for chemical reactions, pharmaceutical compositions, chemical catalysts, polymers, materials science, metallurgy, and other inventions related to the field of chemistry.
Mechanical Engineering; Lightning; Heating; Weapons; Blasting: This category encompasses inventions related to mechanical engineering, lightning protection, heating systems, weapons, and blasting technologies. It includes patents for machines, mechanical devices, heating systems, firearms, explosives, and other inventions related to mechanical engineering and related fields.
Fixed Constructions: This category includes inventions related to the construction industry and built infrastructure. It covers patents for building materials, construction methods, architectural designs, civil engineering projects, and other inventions related to fixed constructions such as buildings, bridges, roads, dams, and other infrastructure projects.

User input will contain a patent abstract.
Classify this abstract to only ONE of the above mentioned categories.
While you assign a category think through carefully and look for a clear conceptual match between the abstract and the label.
Your answer should contain only the numeric label for the category as described above.
"""

In [47]:
few_shot_prompt = [{'role':'system', 'content': system_message}]

In [48]:
for index, row in train_data.to_pandas().sample(4).iterrows():
    example_abstract = row.iloc[0]
    example_label = row.iloc[1]

    few_shot_prompt.append(
        {
            'role': 'user',
            'content': example_abstract
        }
    )

    few_shot_prompt.append(
        {
            'role': 'assistant',
            'content': example_label
        }
    )

In [49]:
predictions, ground_truths = [], []

In [50]:
for index, row in test_data.to_pandas().iterrows():
    gold_abstract = row.iloc[0]
    gold_label = row.iloc[1]

    user_input = [{'role':'user', 'content': gold_abstract}]

    try:
        response = llm.create_chat_completion(
            messages=few_shot_prompt + user_input,
            max_tokens=2,
            temperature=0,
            top_p=0.95,
            min_p=0.05,
            repeat_penalty=1.2,
            mirostat_mode=2
        )

        prediction = int(response['choices'][0]['message']['content'])

        predictions.append(prediction)
        ground_truths.append(gold_label)
    except Exception as e:
        print(e) # Log error and continue
        continue

In [54]:
len(predictions), len(ground_truths)

(5000, 5000)

In [51]:
(np.array(predictions) == np.array(ground_truths)).mean()

0.2978